In [1]:
import tensorflow as tf

'''
Want:
in [1, 0, 1]
Out bool connected/-connected
'''

class GRUCell(object):
    def __init__(self, state_size):
        self._state_size = state_size
        
    @property
    def state_size(self):
        return self._state_size
    
#     def zero_state(self, batch_size):
#         return 
        
    def __call__(self, x, s):
        with tf.name_scope('GRUCell'):
            # x = input vector, s = state vector
            # u = update, r = reset gates
            # c = candidate, h = new state
            # U, W = matrices, b = vector to be learned
            
            u = tf.sigmoid(tf.matmul(tf.matmul(U[0], s) + b[0])
            r = tf.sigmoid(tf.matmul(tf.matmul(U[1], s) + b[1])
            
            c = tf.tanh(tf.matmul(W[2], x) + tf.matmul(U[2], (s * r)) + b[2])
            h = u * s + (1 - u) * c
            return h, h

        
class ca_rnn(object):
    def __init__(self,
                 state_size=4,
                 batch_size=32):

        # x_t [1, 1, 1]
        x = tf.placeholder(tf.float32, shape=[state_size, 1], name='x_t')
        
        # each cell on the board is an input
        inputs = 
        
        with tf.variable_scope('weights') as scope:
            U = tf.get_variable('U', [3, state_size, state_size])
            W = tf.get_variable('W', [3, state_size, state_size])
            b = tf.get_variable('b', [3, state_size], initializer=tf.constant_initializer(1.0))

        zero_state = tf.zeros([batch_size, state_size])
        cell = GRUCell(state_size)

        outputs = []
        for t, input_ in enumerate(inputs):
            if t > 0: 
                scope.reuse_variables()
                
            output, state = cell(input_, state)
            outputs.append(output)
        
def train_network(model, global_steps):
    
    tf.reset_default_graph()
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)

        writer = tf.summary.FileWriter('./graphs/run1', sess.graph)

    #     print(sess.run(c, feed_dict={a: [1, 1]} ))

        writer.close()


model = ca_rnn(4, 1)
train_network(model, 5)